In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import tree

import six
import sys
sys.modules['sklearn.externals.six'] = six

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

from sklearn.ensemble import RandomForestClassifier

In [2]:
# Reading the excel file Cart_file.xlsx - enter the path of the file
df = pd.read_csv(r'OTR_EFS_final_CSV.csv')
print("Number of records: ",len(df))
print("Number of columns: ", len(df.columns))

Number of records:  82852
Number of columns:  79


In [3]:
df.head(5)

,run_date,ar_number,customer_id,creditline_id,wex_national_id,account_open_date,wex_tenure,wex_tenure_group,platform,wex_lob,...,Event_Rate,cycle_group,program_type,employees,bankruptcy__found,bankruptcy__count,out_of__business__indicator,fss_class___03_months_ago,ccs__class___03_months_ago,paydex___03__months_ago
0,2021-10-01 00:00:00,3770001908397,767475,3770001908397,blank,2019-05-23 00:00:00,29,>24M,EFSLLC,OTR,...,0,Weekly,RTS Fleet One,NaN,N,NaN,N,3.0,2.0,80.0
1,2021-10-01 00:00:00,3770017717762,717762,3770017717762,blank,1999-05-07 00:00:00,269,>24M,EFSLLC,OTR,...,0,Daily,NaN,14.0,N,NaN,N,3.0,2.0,80.0
2,2021-10-01 00:00:00,3770017732732,732732,3770017732732,blank,2017-04-07 00:00:00,54,>24M,EFSLLC,OTR,...,0,Weekly,NaN,NaN,N,NaN,N,4.0,3.0,NaN
3,2021-10-01 00:00:00,3770017707857,707857,3770017707857,blank,2014-01-22 00:00:00,93,>24M,EFSLLC,OTR,...,0,Weekly,NaN,7.0,N,NaN,N,2.0,2.0,80.0
4,2021-10-01 00:00:00,3770017703492,703492,3770017703492,blank,2013-05-27 00:00:00,101,>24M,EFSLLC,OTR,...,0,Weekly,RTS Fleet One,4.0,N,NaN,N,4.0,5.0,67.0


In [4]:
#dropping the non required columns
drop_list = ['run_date',
            'ar_number',
            'customer_id',
            'creditline_id',
            'wex_national_id',
            'account_open_date',
            'wex_tenure_group',
            'platform',
            'wex_lob',
            'account_name',
            'arm_status',
            'credit_status',
            'credit_currency',
            'prefunded_accts_flg',
            'open_ar_total',
            'min_ar_date',
            'chargeoff_date',
            'Inactive',
            'nonfuel_spend_bkt',
            'name',
            'years_in_business',
            'years__in__business',
            'd_u_n_s__number',
            'business__name',
            'prediction',
            'behavioral_score',
            'granual_joinid',
            'allrecords',
            'bankruptcy__count',
            'biling_cycle',
            'billing_terms',
            'max_exposure',
            'min_90_dpd',
            'chargeoff_amount']

df.drop(drop_list, axis=1, inplace=True)
print("Number of records: ",len(df))
print("Number of columns: ", len(df.columns))

Number of records:  82852
Number of columns:  45


In [5]:
#convert float function
def convert_float(val):
    try:
        return float(val)
    except ValueError:
        return np.nan

In [6]:
#converting columns into numerical
numerical_cols = ['wex_tenure',
'credit_limit',
'credit_limit_usd',
'available_credit_amount',
'available_credit_limit_usd',
'open_ar_total_usd',
'past_due_days',
'sum_ttl_spend_12m',
'sum_fuel_amt_12m',
'sum_cash_amt_12m',
'sum_misc_amt_12m',
'sum_original_amt_12m',
'sum_amt_used_12m',
'sum_fee_amt_12m',
'total_spend',
'money_code_to_ttl_spend',
'nonfuel_to_ttl_card_spend',
'number_of_trucks__c',
'YIB_Arbtr',
'dnb_hit_flg',
'paydex',
'financial__stress__score',
'commercial__credit__score',
'number_of__negative__payments',
'number_of__satisfactory__payments',
'number_of__slow__payments',
'employees',
'paydex___03__months_ago']

for colname in numerical_cols:
    print(colname)
    df[colname] = df[colname].apply(lambda x: convert_float(x))

wex_tenure
credit_limit
credit_limit_usd
available_credit_amount
available_credit_limit_usd
open_ar_total_usd
past_due_days
sum_ttl_spend_12m
sum_fuel_amt_12m
sum_cash_amt_12m
sum_misc_amt_12m
sum_original_amt_12m
sum_amt_used_12m
sum_fee_amt_12m
total_spend
money_code_to_ttl_spend
nonfuel_to_ttl_card_spend
number_of_trucks__c
YIB_Arbtr
dnb_hit_flg
paydex
financial__stress__score
commercial__credit__score
number_of__negative__payments
number_of__satisfactory__payments
number_of__slow__payments
employees
paydex___03__months_ago


In [7]:
#df['sum_misc_amt_12m'].isnull().value_counts()

In [8]:
#df['sum_misc_amt_12m'].isna().value_counts()

In [7]:
#handling the Missing Values - Categorical Columns
df['has_personal_guarantor'] = df['has_personal_guarantor'].replace(np.nan,'N')
df['chargeoff_flg'] = df['chargeoff_flg'].replace(np.nan,'N')
df['risk_grade__c'] = df['risk_grade__c'].replace(np.nan,'Missing')
df['risk_grade_source__c'] = df['risk_grade_source__c'].replace(np.nan,'Missing')
df['fss__class__score'] = df['fss__class__score'].replace(np.nan,'Missing')
df['commercial__credit__score__class'] = df['commercial__credit__score__class'].replace(np.nan,'Missing')
df['ai_rg'] = df['ai_rg'].replace(np.nan,'Missing')
df['program_type'] = df['program_type'].replace(np.nan,'Missing')
df['bankruptcy__found'] = df['bankruptcy__found'].replace(np.nan,'N')
df['out_of__business__indicator'] = df['out_of__business__indicator'].replace(np.nan,'N')
df['fss_class___03_months_ago'] = df['fss_class___03_months_ago'].replace(np.nan,'Missing')
df['ccs__class___03_months_ago'] = df['ccs__class___03_months_ago'].replace(np.nan,'Missing')

In [8]:
#handling the Missing Values - Numerical Columns
df['sum_cash_amt_12m'] = df['sum_cash_amt_12m'].replace(np.nan,0)
df['sum_original_amt_12m'] = df['sum_original_amt_12m'].replace(np.nan,0)
df['sum_amt_used_12m'] = df['sum_amt_used_12m'].replace(np.nan,0)
df['sum_fee_amt_12m'] = df['sum_fee_amt_12m'].replace(np.nan,0)
df['sum_fuel_amt_12m'] = df['sum_fuel_amt_12m'].replace(np.nan,0)
df['sum_misc_amt_12m'] = df['sum_misc_amt_12m'].replace(np.nan,0)
df['sum_ttl_spend_12m'] = df['sum_ttl_spend_12m'].replace(np.nan,0)
df['nonfuel_to_ttl_card_spend'] = df['nonfuel_to_ttl_card_spend'].replace(np.nan,0)
df['number_of_trucks__c'] = df['number_of_trucks__c'].replace(np.nan,0)

df['paydex'] = df['paydex'].replace(np.nan,50) #paydex
df['financial__stress__score'] = df['financial__stress__score'].replace(np.nan,1450) #fss score
df['commercial__credit__score'] = df['commercial__credit__score'].replace(np.nan,450) #css score

df['number_of__negative__payments'] = df['number_of__negative__payments'].replace(np.nan,0)
df['number_of__satisfactory__payments'] = df['number_of__satisfactory__payments'].replace(np.nan,0)
df['number_of__slow__payments'] = df['number_of__slow__payments'].replace(np.nan,0)
df['employees'] = df['employees'].replace(np.nan,0)

df['paydex___03__months_ago'] = df['paydex___03__months_ago'].replace(np.nan,50) #paydex_3mo_ago

In [9]:
df.to_csv(r'Output\op_missing_value_correction.csv')

In [10]:
categorical_cols = ['issuer_name',
                    'customer_status',
                    'has_personal_guarantor',
                    'has_security_deposit',
                    'chargeoff_flg',
                    'risk_grade__c',
                    'risk_grade_source__c',
                    'fss__class__score',
                    'commercial__credit__score__class',
                    'ai_rg',
                    'cycle_group',
                    'program_type',
                    'bankruptcy__found',
                    'out_of__business__indicator',
                    'fss_class___03_months_ago',
                    'ccs__class___03_months_ago']
print("Number of columns in the Starting:",len(df.columns))
df_OHE = pd.get_dummies(df, columns = categorical_cols)
print("Number of columns in the End:",len(df_OHE.columns))

Number of columns in the Starting: 45
Number of columns in the End: 252


In [11]:
df_OHE.to_csv(r'Output\op_OHE.csv')

In [12]:
df_OHE.head(5)

,wex_tenure,credit_limit,credit_limit_usd,available_credit_amount,available_credit_limit_usd,open_ar_total_usd,past_due_days,sum_ttl_spend_12m,sum_fuel_amt_12m,sum_cash_amt_12m,...,fss_class___03_months_ago_4.0,fss_class___03_months_ago_5.0,fss_class___03_months_ago_Missing,ccs__class___03_months_ago_0.0,ccs__class___03_months_ago_1.0,ccs__class___03_months_ago_2.0,ccs__class___03_months_ago_3.0,ccs__class___03_months_ago_4.0,ccs__class___03_months_ago_5.0,ccs__class___03_months_ago_Missing
0,29.0,20000.0,20000.0,15955.41,15955.41,4044.59,0.0,404795.87,404098.10,471.0,...,0,0,0,0,0,1,0,0,0,0
1,269.0,5000.0,5000.0,2477.84,2477.84,2084.65,0.0,297361.84,296751.77,570.0,...,0,0,0,0,0,1,0,0,0,0
2,54.0,7000.0,7000.0,6318.55,6318.55,687.45,0.0,36614.28,36151.70,0.0,...,1,0,0,0,0,0,1,0,0,0
3,93.0,15000.0,15000.0,11587.48,11587.48,3412.52,0.0,348743.44,348743.44,0.0,...,0,0,0,0,0,1,0,0,0,0
4,101.0,5000.0,5000.0,3810.56,3810.56,1197.44,0.0,91919.43,91919.43,0.0,...,1,0,0,0,0,0,0,0,1,0


In [13]:
df_OHE.isna().sum()

wex_tenure                            0
credit_limit                          0
credit_limit_usd                      0
available_credit_amount               0
available_credit_limit_usd            0
                                     ..
ccs__class___03_months_ago_2.0        0
ccs__class___03_months_ago_3.0        0
ccs__class___03_months_ago_4.0        0
ccs__class___03_months_ago_5.0        0
ccs__class___03_months_ago_Missing    0
Length: 252, dtype: int64

In [14]:
#checking if data is finite - this should be true
print('Checking if data is finite- Excluding V2 Variables:', np.all(np.isfinite(df_OHE)))

#checking if there are NA - this should be false
print('Checking if data have NA- Excluding V2 Variables:',np.all(np.isnan(df_OHE)))

Checking if data is finite- Excluding V2 Variables: True
Checking if data have NA- Excluding V2 Variables: False


# Attempting CART - Fitting the Tree Algorithm

In [15]:
%%time
#Test-Train Sample Creations
# Defining X and y - Train (90%) and Test (10%)
X = df_OHE.drop('Event_Rate',axis=1)
y = df_OHE[['Event_Rate']]

# Train test splitting
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=0,stratify=y)
print(X_train.shape,X_test.shape)

(74566, 251) (8286, 251)
Wall time: 1.54 s


In [22]:
%%time 
## Wall time: 8min 32s
#Running the Gridsearch to check best parameters
# Setting weights for class imbalance - running gridsearch with 10K samples
weights = {0:1, 1:11}
dtree  = DecisionTreeClassifier(class_weight=weights, random_state=0)
# Defining the parameters for parameter tuning
params = {'max_depth': [3,4,5,6,7,8,9,10,11,12,13],
         'min_samples_leaf' : [3,4,5,8],
         'min_samples_split':[3,4,5,8],
         'criterion': ["gini","entropy"]}
# Fitting the GrdiSearchCV tuning algorithm
cv = GridSearchCV(dtree, param_grid = params, scoring = 'roc_auc', cv = 5, n_jobs=-1)
cv.fit(X_train, y_train)

Wall time: 14min 20s


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight={0: 1, 1: 11},
                                              random_state=0),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                         'min_samples_leaf': [3, 4, 5, 8],
                         'min_samples_split': [3, 4, 5, 8]},
             scoring='roc_auc')

In [34]:
#Calculating the Importance Matrix
# Creating a custom function
def create_dt_importance(X_train,y_train, depth, top_n_feats, imp_dict={}):
    dtc = DecisionTreeClassifier(max_depth = depth,
                                   min_samples_leaf = cv.best_params_['min_samples_leaf'],
                                   min_samples_split = cv.best_params_['min_samples_split'],
                                  class_weight=cv.best_estimator_.class_weight,
                                  random_state=cv.best_estimator_.random_state,
                                 criterion=cv.best_estimator_.criterion )
    dtc.fit(X_train,y_train)
    y_pred_f = dtc.predict(X_test)
    y_prob = dtc.predict_proba(X_test)
    
    #Calculating feature importance curve
    importance = dtc.feature_importances_
    
    #Storing the importance scores in the importance dictionary
    for i,ii in enumerate(importance):
        imp_dict[X.columns[i]]=importance[i]
        
    # Sort the dictionary
    sorted_dict = sorted(imp_dict.items(),key = lambda kv: kv[1],reverse=True)
    
    return sorted_dict[:top_n_feats]

In [35]:
%%time
importance_matrix = pd.DataFrame(create_dt_importance(X_train,y_train,cv.best_params_['max_depth'],254))
importance_matrix.rename(columns = {0:'variable', 1:'Importance'}, inplace = True)
importance_matrix.to_csv(r'Output\importance_matrix_CART.csv') #setup the file path here

Wall time: 4.06 s


In [36]:
%%time
# Selecting the top n=150 features - updating Test & Train with only selected important features
n=150
X_train_top = X_train[[i[0] for i in create_dt_importance(X_train,y_train, cv.best_params_['max_depth'], n)]]
X_test_top = X_test[[i[0] for i in create_dt_importance(X_train,y_train, cv.best_params_['max_depth'], n)]]
print(X_train_top.shape, X_test_top.shape)

(74566, 150) (8286, 150)
Wall time: 8.07 s


In [37]:
%%time
# Fitting the dtree algo- Attempting CART
dtree  = DecisionTreeClassifier(criterion=cv.best_estimator_.criterion,
                               class_weight=weights, #None not class weight
                               max_depth = cv.best_params_['max_depth'], 
                               min_samples_leaf = cv.best_params_['min_samples_leaf'],
                               min_samples_split = cv.best_params_['min_samples_split'], 
                               random_state=cv.best_estimator_.random_state)
dtree.fit(X_train_top, y_train)
y_pred_f = dtree.predict(X_test_top)
y_prob = dtree.predict_proba(X_test_top)

Wall time: 3.53 s


In [38]:
roc_auc_score(y_test.values.flatten(),y_prob.argmax(axis=-1))

0.6484599036348021

In [19]:
# %%time

# #tree for visulaization
# dtree1  = DecisionTreeClassifier(criterion='gini',
#                                class_weight=weights, #None not class weight
#                                max_depth = 6, 
#                                min_samples_leaf = 2,
#                                min_samples_split = 2, 
#                                random_state=cv.best_estimator_.random_state)
# dtree1.fit(X_train_top, y_train)

# # Visualising the tree graph
# dot_data = StringIO()
# export_graphviz(dtree1, out_file=dot_data, filled=True, rounded=True,special_characters=True,
#                 feature_names = X_train_top.columns,class_names=['Not Default','Default'],precision=0)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_png(r'Output\OP_top50_imp_var.png')
# Image(graph.create_png())

# Attempting Random Forest

In [40]:
%%time

Test_Size = 0.10

#Test-Train Sample Creations
# Defining X and y - Train (90%) and Test (10%)
X = df_OHE.drop('Event_Rate',axis=1)
y = df_OHE[['Event_Rate']]

# Train test splitting
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=Test_Size,random_state=0,stratify=y)
print(X_train.shape,X_test.shape)


(74566, 251) (8286, 251)
Wall time: 1.4 s


In [ ]:
%%time
#Running gridsearch to find best parameters - Train and Test split is already done at CART stage

random_state = 10 #seeding for random state
rf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)

param_grid = { 
    'n_estimators': [50,200,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [7,9,10,12],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'criterion' :['gini']
}


cv_rf = GridSearchCV(estimator=rf, param_grid=param_grid,scoring = 'roc_auc', cv= 5, n_jobs=-1)
cv_rf.fit(X_train.sample(n=10000, random_state=random_state), y_train.sample(n=10000, random_state=random_state))
print(cv_rf.best_params_)

In [ ]:
#Calculating the Importance Matrix
# Creating a custom function
def create_rf_importance(X_train_IP, y_train_IP, X_test_IP, X_IP, cv_IP, top_n_feats, imp_dict={}):
    rfc = RandomForestClassifier(max_depth = cv_IP.best_params_['max_depth'],
                                n_estimators= cv_IP.best_params_['n_estimators'],
                                class_weight=cv_IP.best_estimator_.class_weight,
                                random_state=cv_IP.best_estimator_.random_state,
                                criterion=cv_IP.best_estimator_.criterion ,
                                min_samples_leaf=cv_IP.best_estimator_.min_samples_leaf,
                                min_samples_split= cv_IP.best_estimator_.min_samples_split,
                                n_jobs=-1)
    rfc.fit(X_train_IP,y_train_IP)
    y_pred_f = rfc.predict(X_test_IP)
    y_prob = rfc.predict_proba(X_test_IP)
    
    #Calculating feature importance curve
    importance = rfc.feature_importances_
    
    #Storing the importance scores in the importance dictionary
    for i,ii in enumerate(importance):
        imp_dict[X_IP.columns[i]]=importance[i]
        
    # Sort the dictionary
    sorted_dict = sorted(imp_dict.items(),key = lambda kv: kv[1],reverse=True)
    
    return sorted_dict[:top_n_feats]

In [ ]:
%%time
print('Creating the Important Matrix - final files will be saved in Output folder')

importance_matrix_rf = pd.DataFrame(create_rf_importance(X_train,y_train,X_test,X,cv_rf,254))
importance_matrix_rf.rename(columns = {0:'variable', 1:'Importance'}, inplace = True)
importance_matrix_rf.to_csv(r'Output\importance_matrix_rf.csv')


In [ ]:
%%time
print('Selecting Top N features for Random Forest Plotting')
n=150
# Selecting the top n features - updating Test & Train with only selected important features
X_train_top = X_train[[i[0] for i in create_rf_importance(X_train,y_train,X_test,X,cv_rf,n)]]
X_test_top = X_test[[i[0] for i in create_rf_importance(X_train,y_train,X_test,X,cv_rf,n)]]
print(X_train_top.shape, X_test_top.shape)

In [ ]:
%%time
weights_rf = {0:1, 1:50}
rf1 = RandomForestClassifier(n_estimators=cv_rf.best_params_['n_estimators'],
                             max_depth= cv_rf.best_params_['max_depth'] ,
                             max_features= cv_rf.best_params_['max_features'],
                             class_weight= weights_rf,
                             criterion=cv_rf.best_estimator_.criterion,
                             random_state=cv_rf.best_estimator_.random_state,
                             n_jobs=-1)
rf1.fit(X_train_top, y_train)

y_pred_f1 = rf1.predict(X_test_top)
y_prob1 = rf1.predict_proba(X_test_top)

In [ ]:

print('\n------------------------------------------------\n')
print('ROC', roc_auc_score(y_test.values.flatten(),y_prob1.argmax(axis=-1)))
print('------------------------------------------------')
print('Classification Report \n',classification_report(y_test,y_pred_f1))


In [ ]:
%%time

#rf for visulaization
rf2 = RandomForestClassifier(n_estimators=5,
                             max_depth= 6 ,
                             max_features= cv_rf.best_params_['max_features'],
                             class_weight= weights_rf,
                             criterion='gini',
                             random_state=cv_rf.best_estimator_.random_state,
                             n_jobs=-1)
rf2.fit(X_train_top, y_train)



from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

fn=X_train_top.columns
cn=['Not Fraud','Fraud']
fig, axes = plt.subplots(nrows = 1,ncols = 1, figsize = (10,10), dpi=3000)
# Export as dot file
dot_data = export_graphviz(rf2.estimators_[0], feature_names = fn,class_names = cn,
                rounded = True, proportion = False, 
                precision = 2, filled = True, out_file=None)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

graph = graph_from_dot_data(dot_data)
graph.write_png(r'Output\RF_OP_Excluding_V2_Variables_0.png')
Image(graph.create_png())


### Attempting GBM

In [86]:
%%time
#Train (70%) and Test (30%)
X = df_OHE.drop('Event_Rate',axis=1)
y = df_OHE[['Event_Rate']]


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=0,stratify=y)
print(X_train.shape,X_test.shape)

(57996, 251) (24856, 251)
Wall time: 326 ms


In [87]:
from sklearn.ensemble import GradientBoostingClassifier

In [130]:
y['Event_Rate'].value_counts() # Imbalanced dataset

0    80811
1    2041 
Name: Event_Rate, dtype: int64

In [144]:
def func(list1=[]):
    for value in df['Event_Rate']:
        if value == 0:
            list1.append(0.5)
        else:
            list1.append(4)
    return list1
x=func()
print(len(x))

82852


In [147]:
#x[:500]

In [ ]:
%%time


random_state = 5 

gbm = GradientBoostingClassifier(n_estimators=100, random_state=random_state)



param_grid = { 
    'n_estimators': [50,70,80,90,100],
    'learning_rate': [0.01,0.03,0.05,0.1,0.15],
    'max_depth' : [5,7,10,12,14],
    'min_samples_split': [2, 5, 7,9,10],
    'criterion':['squared_error','friedman_mse','mse']
    
    
}

# need solution for sample weight and .sample method
cv_rf = GridSearchCV(estimator=gbm, param_grid=param_grid,scoring = 'roc_auc', cv= 2, n_jobs=-1)
cv_rf.fit(X_train.sample(n=50000, random_state=random_state), y_train.sample(n=50000, random_state=random_state),sample_weight=x[:50000])
#print(cv_rf.best_params_)

In [157]:
cv_rf.best_params_

{'criterion': 'squared_error',
 'learning_rate': 0.05,
 'max_depth': 5,
 'min_samples_split': 9,
 'n_estimators': 90}

In [158]:
cv_rf.best_params_['n_estimators']

90

In [162]:
%%time

gbm2 = GradientBoostingClassifier(learning_rate=0.05,
                             n_estimators=cv_rf.best_params_['n_estimators'],
                             random_state=cv_rf.best_estimator_.random_state,
                             max_depth= cv_rf.best_params_['max_depth'] ,
                             
                             criterion=cv_rf.best_estimator_.criterion,
                             min_samples_split = cv_rf.best_params_['min_samples_split']
                             
                             )
gbm2.fit(X_train, y_train,sample_weight=x[:57996])

y_pred_f1 = gbm2.predict(X_test)
y_prob1 = gbm2.predict_proba(X_test)

Wall time: 53.1 s


In [163]:
print('\n------------------------------------------------\n')
print('ROC', roc_auc_score(y_test.values.flatten(),y_prob1.argmax(axis=-1)))
print('------------------------------------------------')
print('Classification Report \n',classification_report(y_test,y_pred_f1))



------------------------------------------------

ROC 0.5231565953115009
------------------------------------------------
Classification Report 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     24244
           1       0.53      0.05      0.09       612

    accuracy                           0.98     24856
   macro avg       0.75      0.52      0.54     24856
weighted avg       0.97      0.98      0.97     24856



In [92]:
#Importance Matrix

def create_rf_importance(X_train_IP, y_train_IP, X_test_IP, X_IP, cv_IP, top_n_feats, imp_dict={}):
    gbmClassifier = GradientBoostingClassifier(learning_rate=0.03,
                                max_depth = cv_IP.best_params_['max_depth'],
                                n_estimators= cv_IP.best_params_['n_estimators'],
                                random_state=5,
                                criterion=cv_IP.best_estimator_.criterion ,
                                max_features= cv_rf.best_params_['max_features'],
                                min_samples_split= cv_IP.best_estimator_.min_samples_split,
                                )
    gbmClassifier.fit(X_train_IP,y_train_IP)
    y_pred_f = gbmClassifier.predict(X_test_IP)
    y_prob = gbmClassifier.predict_proba(X_test_IP)
    
    #Calculating feature importance curve
    importance = gbmClassifier.feature_importances_
    
    #Storing the importance scores in the importance dictionary
    for i,ii in enumerate(importance):
        imp_dict[X_IP.columns[i]]=importance[i]
        
    # Sort the dictionary
    sorted_dict = sorted(imp_dict.items(),key = lambda kv: kv[1],reverse=True)
    
    return sorted_dict[:top_n_feats]

In [93]:
%%time
print('Creating the Important Matrix - final files will be saved in Output folder')

importance_matrix_rf = pd.DataFrame(create_rf_importance(X_train,y_train,X_test,X,cv_rf,100))
importance_matrix_rf.rename(columns = {0:'variable', 1:'Importance'}, inplace = True)
importance_matrix_rf.to_csv(r'Output\importance_matrix_GBM.csv')


Creating the Important Matrix - final files will be saved in Output folder
Wall time: 7.15 s


In [94]:
importance_matrix_rf

,variable,Importance
0,sum_ttl_spend_12m,0.059755
1,sum_fuel_amt_12m,0.054981
2,total_spend,0.054007
3,nonfuel_to_ttl_card_spend,0.052274
4,wex_tenure,0.044051
...,...,...
95,program_type_New EFS Promotion 2021,0.001029
96,has_security_deposit_Y,0.001000
97,commercial__credit__score__class_1.0,0.000969
98,risk_grade__c_0.0,0.000940


In [115]:
len(np.zeros(len(y_train)))

57996

In [121]:
sample_weights = np.zeros(len(y_train))
sample_weights[y_train == 0] = 0.5
sample_weights[y_train == 1] = 9.1

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

0    80811
1    2041 
Name: Event_Rate, dtype: int64